# My Model

## 1. data prepare

In [1]:
from scripts.data_reader import read_train_data,read_test_data
from scripts.evaluate import evaluate
from config import config
import pandas as pd
import anndata as ad

In [2]:
input_train_mod1, input_train_mod2 = read_train_data(config)
input_test_mod1, input_test_mod2 = read_test_data(config)

## 2. train data

In [3]:
input_train = ad.concat(
    {"train": input_train_mod1, "test": input_test_mod1},
    axis=0,
    join="outer",
    label="group",
    fill_value=0,
    index_unique="-"
)

In [4]:
input_train

AnnData object with n_obs × n_vars = 500 × 600
    obs: 'batch', 'size_factors', 'group'
    layers: 'counts'

In [5]:
input_train_mod2

AnnData object with n_obs × n_vars = 221 × 600
    obs: 'batch', 'size_factors'
    var: 'feature_types'
    uns: 'dataset_id', 'gene_activity_var_names', 'organism'
    obsm: 'gene_activity'
    layers: 'counts'

## 3. VAE reduction

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [7]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        # 编码器
        self.fc1 = nn.Linear(input_dim, 400)
        self.fc21 = nn.Linear(400, latent_dim)
        self.fc22 = nn.Linear(400, latent_dim)
        # 解码器
        self.fc3 = nn.Linear(latent_dim, 400)
        self.fc4 = nn.Linear(400, input_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, input_dim))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar, z

def vae_loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) # KL散度
    return BCE + KLD

In [8]:
# 假设 input_train 和 input_train_mod2 是 NumPy 数组或 Pandas DataFrame
tensor_x = torch.Tensor(input_train.X.toarray())  # 转换 input_train
tensor_y = torch.Tensor(input_train_mod2.X.toarray())      # 转换 input_train_mod2

# 创建数据加载器
dataset_x = TensorDataset(tensor_x) 
dataset_y = TensorDataset(tensor_y)

loader_x = DataLoader(dataset_x, batch_size=32, shuffle=True)
loader_y = DataLoader(dataset_y, batch_size=32, shuffle=True)

In [9]:
# 设定输入维度和潜在空间维度
input_dim = input_train_mod1.shape[1] 
input_dim_x = input_train.shape[1]
input_dim_y = input_train_mod2.shape[1]
latent_dim = 50  # 潜在空间维度

# 初始化两个模型
model_x = VAE(input_dim_x, latent_dim)
model_y = VAE(input_dim_y, latent_dim)

# 训练模型（以 model_x 为例）
optimizer_x = torch.optim.Adam(model_x.parameters(), lr=1e-3)
epochs = 10  # 迭代次数

for epoch in range(epochs):
    model_x.train()
    train_loss = 0
    for batch_idx, (data,) in enumerate(loader_x):
        optimizer_x.zero_grad()
        recon_batch, mu, logvar, z = model_x(data)
        loss = vae_loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer_x.step()
    print(f'Epoch {epoch}, Loss: {train_loss / len(loader_x.dataset)}')

optimizer_y = torch.optim.Adam(model_y.parameters(), lr=1e-3)
epochs = 10  # 迭代次数

# 对 model_y 重复上述训练过程
for epoch in range(epochs):
    model_y.train()
    train_loss = 0
    for batch_idx, (data,) in enumerate(loader_y):
        optimizer_y.zero_grad()
        recon_batch, mu, logvar, z = model_y(data)
        loss = vae_loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer_y.step()
    print(f'Epoch {epoch}, Loss: {train_loss / len(loader_x.dataset)}')

Epoch 0, Loss: 197.69783837890625
Epoch 1, Loss: 159.724333984375
Epoch 2, Loss: 156.34261767578124
Epoch 3, Loss: 155.1858876953125
Epoch 4, Loss: 154.1045732421875
Epoch 5, Loss: 153.31967138671874
Epoch 6, Loss: 152.47911669921876
Epoch 7, Loss: 151.53439892578126
Epoch 8, Loss: 150.872818359375
Epoch 9, Loss: 150.251142578125
Epoch 0, Loss: 55.94428369140625
Epoch 1, Loss: 36.80267626953125
Epoch 2, Loss: 29.672128662109376
Epoch 3, Loss: 27.81671630859375
Epoch 4, Loss: 27.03416650390625
Epoch 5, Loss: 26.806733154296875
Epoch 6, Loss: 26.699676513671875
Epoch 7, Loss: 26.542757568359374
Epoch 8, Loss: 26.499392578125
Epoch 9, Loss: 26.44298291015625


In [10]:
model_x.eval()
with torch.no_grad():
    latent_x = [model_x.encode(data) for data, in loader_x]

# 对 model_y 重复上述降维过程
model_y.eval()
with torch.no_grad():
    latent_y = [model_y.encode(data.view(-1, input_dim_y))[0].numpy() for data, in loader_y]


In [11]:
tensors_x = []
with torch.no_grad():
    for idx, [x] in enumerate(loader_x):
        #x = x.to(device)
        _, _, _, z = model_x(x)
        tensors_x.append(z)

concatenated_tensor_x = torch.cat(tensors_x, dim=0)

In [12]:
concatenated_tensor_x.shape

torch.Size([500, 50])

In [13]:
tensors_y = []
with torch.no_grad():
    for idx, [y] in enumerate(loader_y):
        #x = x.to(device)
        _, _, _, z = model_y(y)
        tensors_y.append(z)

concatenated_tensor_y = torch.cat(tensors_y, dim=0)

In [14]:
concatenated_tensor_y.shape

torch.Size([221, 50])

## 4. Linear regression

In [15]:
from sklearn.ensemble import RandomForestRegressor

In [16]:
X_train = concatenated_tensor_x[input_train.obs['group'] == 'train']
X_test = concatenated_tensor_x[input_train.obs['group'] == 'test']
y_train = concatenated_tensor_y

In [17]:
assert len(X_train) + len(X_test) == len(concatenated_tensor_x)

In [18]:
reg = RandomForestRegressor()

# Train the model on the PCA reduced modality 1 and 2 data
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [19]:
y_pred.shape

(279, 50)

## 5. Prediction

In [20]:
tensor_y_pred = torch.Tensor(input_test_mod1.X.toarray()) 
dataset_y_pred = TensorDataset(tensor_y_pred)
loader_y_pred = DataLoader(dataset_y_pred, batch_size=32, shuffle=True)

In [21]:
tensors = []
with torch.no_grad():
    for idx, [y_pred] in enumerate(loader_y_pred):
        #x = x.to(device)
        recon_y_pred, _, _, _ = model_y(y_pred)
        tensors.append(recon_y_pred)

concatenated_tensor = torch.cat(tensors, dim=0)

In [22]:
concatenated_tensor.numpy().shape

(279, 600)

## 6. Save

In [23]:
from scipy.sparse import csc_matrix

In [24]:
prediction = csc_matrix(concatenated_tensor.numpy())

if config["save_predict"]:
    out = ad.AnnData(
        X=prediction,
        obs=input_test_mod1.obs,
        var=input_train_mod2.var,
        uns={
            'dataset_id': input_train_mod1.uns['dataset_id'],
            'method_id': config["method_name"],
        },
    )
    out.write_h5ad(config["output"], compression="gzip")

## 7. Evaluate

In [25]:
from scripts.evaluate import evaluate
from config import config

In [26]:
if config["do_test"]:
    result = evaluate(prediction, input_test_mod2.X)

if config["write_result"]:
    pd.DataFrame([result]).to_csv(config["output"].replace("h5ad","csv"))

RMSE: 0.32230380177497864
MAE: 0.21604646742343903
